In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
pretrained_mnist_model = tf.keras.models.load_model("/content/drive/MyDrive/pretrained_mnist_model.h5")

In [7]:
pretrained_mnist_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
__________________________________________________

In [8]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten: True
dense: True
leaky_re_lu: True
dense_1: True
leaky_re_lu_1: True
dense_2: True


In [9]:
for layer in pretrained_mnist_model.layers[:-1]:
  layer.trainable = False
  print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
leaky_re_lu: False
dense_1: False
leaky_re_lu_1: False


In [10]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
leaky_re_lu: False
dense_1: False
leaky_re_lu_1: False
dense_2: True


In [14]:
lower_pretrained_layers = pretrained_mnist_model.layers[:-1]

new_model = tf.keras.models.Sequential(lower_pretrained_layers)
new_model.add(tf.keras.layers.Dense(2, activation="softmax"))

In [15]:
new_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 265,802
Trainable params: 202
Non-trainable params: 265,600
______________________________________________

In [16]:
def update_even_odd_labels(labels):
  for idx, label in enumerate(labels):
    labels[idx] = np.where(label % 2 == 0, 1, 0)
  return labels

In [17]:
y_train_bin, y_test_bin, y_valid_bin = update_even_odd_labels([y_train, y_test, y_valid])

In [18]:
new_model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [19]:
history = new_model.fit(X_train, y_train_bin, epochs=10,
                    validation_data=(X_valid, y_valid_bin), verbose=2)

Epoch 1/10
1719/1719 - 5s - loss: 0.4190 - accuracy: 0.8107 - val_loss: 0.3363 - val_accuracy: 0.8578
Epoch 2/10
1719/1719 - 2s - loss: 0.3431 - accuracy: 0.8527 - val_loss: 0.3111 - val_accuracy: 0.8702
Epoch 3/10
1719/1719 - 2s - loss: 0.3251 - accuracy: 0.8606 - val_loss: 0.2990 - val_accuracy: 0.8748
Epoch 4/10
1719/1719 - 2s - loss: 0.3147 - accuracy: 0.8662 - val_loss: 0.2900 - val_accuracy: 0.8806
Epoch 5/10
1719/1719 - 2s - loss: 0.3072 - accuracy: 0.8695 - val_loss: 0.2838 - val_accuracy: 0.8842
Epoch 6/10
1719/1719 - 2s - loss: 0.3013 - accuracy: 0.8718 - val_loss: 0.2786 - val_accuracy: 0.8848
Epoch 7/10
1719/1719 - 2s - loss: 0.2965 - accuracy: 0.8748 - val_loss: 0.2744 - val_accuracy: 0.8884
Epoch 8/10
1719/1719 - 2s - loss: 0.2924 - accuracy: 0.8767 - val_loss: 0.2706 - val_accuracy: 0.8900
Epoch 9/10
1719/1719 - 2s - loss: 0.2890 - accuracy: 0.8784 - val_loss: 0.2673 - val_accuracy: 0.8918
Epoch 10/10
1719/1719 - 2s - loss: 0.2860 - accuracy: 0.8800 - val_loss: 0.2650 - 

In [20]:
new_model.evaluate(X_test, y_test_bin)

313/313 [==============================] - 1s 2ms/step - loss: 0.2798 - accuracy: 0.8845


[0.2797868251800537, 0.8845000267028809]

In [21]:
X_new = X_test[:3]

y_test[:3], y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([0, 1, 0]))

In [22]:
np.argmax(new_model.predict(X_new), axis=-1)

array([0, 1, 0])

In [23]:
new_model.save("/content/drive/MyDrive/mnist_model_odd_even.h5")